### Part one: use synthetic graphs

In the first part, you can use some graphs obtained by using the networx library or the library of your choice.

For each selected graph (max 2 or 3) you can perform different types of attack: turn off nodes at random, turn off the highest degree nodes, those with the highest pagerank, those with the highest clustering coefficient, ...

After each removal, compute new measures, for example the size of the giant component or the diameter of the network and then plot these measures with respect to node failures.

Be careful, some of the networkx functions you will use work only for undirected, connected graphs and therefore you need to instrument your code to work on the entire graph first, and on the several components after the split of the original graph into smaller clusters.


In [1]:
import networkx
#graph = networkx.read_edgelist("../graphs/p2p-Gnutella31.txt")
graph = networkx.read_edgelist("../graphs/power-grid.txt")
#graph = networkx.gnp_random_graph(200, 0.1)


TypeError: Failed to convert edge data (['24']) to dictionary.

In [0]:
# Import functions
import random

def get_highest_degree_node(G):
    return sorted(G.degree, key=lambda x: x[1], reverse=True)[0][0]

def get_random_node(G):
    list_of_nodes = G.nodes()
    return random.sample(list_of_nodes, 1)[0]

In [0]:
# Analyze relative giant component size (relative to the starting graph size)
def connected_component_subgraphs(G):
    for c in networkx.connected_components(G):
        yield G.subgraph(c)
def get_giant_component_size(G):
    return len(sorted(connected_component_subgraphs(G), key=len, reverse=True)[0])

# Saving inital graph' size
n_nodes = len(graph)

# Creating list variable to iterate
graphs = []
sizes = []
functions = []

# One graph copy and one array of data for each tecnique
random_graph = graph.__class__(graph)
random_giant_sizes = []
graphs.append(random_graph)
sizes.append(random_giant_sizes)
functions.append(get_random_node)

degree_graph = graph.__class__(graph)
degree_giant_sizes = []
graphs.append(degree_graph)
sizes.append(degree_giant_sizes)
functions.append(get_highest_degree_node)

'''
closeness_graph = graph.__class__(graph)
closeness_giant_sizes = []
graphs.append(closeness_graph)
sizes.append(closeness_giant_sizes)

betweenness_graph = graph.__class__(graph)
betweenness_giant_sizes = []
graphs.append(betweenness_graph)
sizes.append(betweenness_giant_sizes)

clustering_graph = graph.__class__(graph)
clustering_giant_sizes = []
graphs.append(clustering_graph)
sizes.append(clustering_giant_sizes)

'''
for i in range(len(graphs)):
        sizes[i].append(get_giant_component_size(graphs[i]) / n_nodes)

for _ in range(len(graph) - 1):
    # Per ogni tipo di attacco
    for i in range(len(graphs)):
        node_to_remove = functions[i](graphs[i])
        graphs[i].remove_node(node_to_remove)
        sizes[i].append(get_giant_component_size(graphs[i]) / n_nodes)


print(degree_giant_sizes)

In [0]:
# Plotting size results
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(0, 1, 1.0/len(graph))

colors = ['r', 'b', 'g', 'y', 'p']
for i in range(len(sizes)): 
    plt.plot(x, sizes[i], colors[i])

plt.show()